This notebook is used to predict completly new molecules.

In [1]:
import deepchem as dc
import numpy as np
import pandas as pd

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,RationalQuadratic, Matern, WhiteKernel, ConstantKernel

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Error: Unable to import pysam. Please make sure it is installed.
Error: Unable to import pysam. Please make sure it is installed.
Error: Unable to import pysam. Please make sure it is installed.

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [4]:
@ignore_warnings(category=ConvergenceWarning)
def main():
    #load the dataset (polymer smiles and a specific task)

    #tasks = ["PC", "TB", "TC"]
    tasks = ["TB"]
    # read the csv file containening the neccesarry rdkit descriptors
    feats_df = pd.read_csv('../csv_files/RFE_features.csv')


    for task in tasks:

        # read the dataset belonging to the rigth property
        print("\n\nprediciting:",task)
        file = '../csv_files/Combined_' + task + '.csv'

        #get the descriptors neccesarry for the property
        descriptors = feats_df[task]
        descriptorlist = descriptors.dropna().to_list()

        print("\nloading the", task, "data...")
        loader = dc.data.CSVLoader([task], feature_field="smiles", featurizer=dc.feat.RDKitDescriptors(descriptorlist))
        Data = loader.create_dataset(file)

        #some RDKit descriptors return nan, make these 0
        X = np.nan_to_num(Data.X, copy=True, nan=0.0, posinf=0)

        #add data to dataset with the according propertie to be predicted
        Dataset = dc.data.DiskDataset.from_numpy(X=X, y=Data.y, w=Data.w, ids=Data.ids, tasks = [task])
        
    
            
        #split the dataset using the random splitter
        splitter = dc.splits.RandomSplitter()
        train_dataset, valid_dataset = splitter.train_test_split(Dataset, frac_train = 0.99)


        # create the GPR model and fit the model
        kernel = 1 * RationalQuadratic() + 1 * WhiteKernel() + 1 * ConstantKernel() * RationalQuadratic()
        model = dc.models.SklearnModel(GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10))
        model.fit(train_dataset)
        
        #calculate r2 scores
        metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
        train_r2score = model.evaluate(train_dataset, metric)
        valid_r2score = model.evaluate(valid_dataset, metric)

        #make then useable and print them
        validr2=list(valid_r2score.values())[0]
        trainr2=list(train_r2score.values())[0]
        print("train-r2:",round(trainr2,3),"valid-r2:", round(validr2,3))

        #load the test set into a list
        SmileList = pd.read_csv('../csv_files/test_data.csv')["smiles"].tolist()

        #featurize the test set
        featurizer = dc.feat.RDKitDescriptors(descriptorlist)
        features = featurizer.featurize(SmileList)
        X_topredict = np.nan_to_num(features, copy=True, nan=0.0)

        #add it to a deepchem datastructure
        test_dataset = dc.data.DiskDataset.from_numpy(X=X_topredict, ids=SmileList, tasks = ["Eat"])

        #predict the test set
        predicted_test = model.predict(test_dataset)

        for idx, smiles in enumerate(SmileList):
            print(smiles,task, ": ",predicted_test[idx])

main()




prediciting: TB

loading the TB data...
train-r2: 1.0 valid-r2: 0.998
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC TB :  544.122848033905
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC TB :  610.8878096938133
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC TB :  537.4652728438377
CCCCO TB :  58.943196415901184
